# Семинар 10: введение в АБ-тесты

Теперь давайте посмотрим, как задачи, которые были заданы вам на дом, решаются в python.

In [1]:
import numpy as np
import pandas as pd

## Задачка 1

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [2]:
mu = 525
sd = 100

n = 100
mean = 541.4

# руками считаем границы доверительного интервала: 
left = (mean - mu) - 2*sd/np.sqrt(n)
right = (mean - mu) + 2*sd/np.sqrt(n)

print('Доверительный интервал: [{:.2f}; {:.2f}]'.format(left,right))

Доверительный интервал: [-3.60; 36.40]


In [3]:
# Теперь то же самое через t-статистику! 

t_nabl = (mean - mu)/(sd/np.sqrt(n))
print(t_nabl)

1.6399999999999977


Как было выше, доверительные интервалы, но с уточнённым значением $1.96$ можно построить внутренними средствами python.

In [4]:
left = (mean - mu) - 1.96*sd/np.sqrt(n)
right = (mean - mu) + 1.96*sd/np.sqrt(n)

print('Доверительный интервал: [{:.2f}; {:.2f}]'.format(left,right))

Доверительный интервал: [-3.20; 36.00]


In [5]:
from statsmodels.stats.weightstats import _zconfint_generic

_zconfint_generic(mean - mu,  sd/np.sqrt(n), 0.05, 'two-sided')

(-3.1996398454005615, 35.999639845400516)

## Задачка 2

Олег подбрасывает монетку и орёт: "ОРЁЛ-РЕШКА-ОРЁЛ-РЕШКА!". Ещё он недавно посмотрел фильм Кристофера Нолана "Тёмный рыцарь". Там ему очень понравился Харви Дент. Потому что у него тоже была монетка, которую тот подбрасывал. ЛСП стало интересно: а правильная ли у него монетка. Действительно ли она выпадает орлом с вероятностью $\frac{1}{2}$? 

* Олег подбросил монетку трижды и получил комбинацию: $OPP$. Найдите долю выпадения орла.  Дальше будем обозначать эту долю как $\hat p$. 
* На теории вероятностей вы докажете, что стандартное отклонение для доли считается по формуле $ \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}}.$ Найдите среднее отклонение доли. 
* Можно показать, что $\hat p$ имеет нормальное распределение. Постройте на основе правила двух сигм для вашей оценки доли доверительный интервал. Найдите его ширину. 
*  Олег подбросил монетку ещё два раза и получил $OPPOP$. Найдите доверительный интервал для этой ситуации. Найдите его ширину. Стал ли он уже? Почему это произошло? 
* Проверьте гипотезу о том, что $p = \frac{1}{2}$ против гипотезы $p \ne \frac{1}{2}$

Строим доверительный интервал для $p$ по формуле:

$$\hat{p}\pm 1.96 \cdot \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [6]:
n = 3
m = 1
p_hat = m/n

std = np.sqrt(p_hat*(1-p_hat)/n)


# руками считаем границы доверительного интервала: 
left =  p_hat - 1.96*std
right = p_hat + 1.96*std

print('Доверительный интервал: [{:.2f}; {:.2f}]'.format(left,right))

Доверительный интервал: [-0.20; 0.87]


In [7]:
from statsmodels.stats.proportion import proportion_confint

# первым аргументом надо указать число удачных подбрасываний, вторым общее число бросков
normal_interval = proportion_confint(m, n, alpha = 0.05)
normal_interval

(0.0, 0.866767964039479)

In [8]:
print('Ширина интервала:', normal_interval[1] - normal_interval[0])

Ширина интервала: 0.866767964039479


Увеличим число наблюдений! 

In [9]:
n = 5
m = 2

normal_interval = proportion_confint(m, n, alpha = 0.05)

print('Ширина интервала:', normal_interval[1] - normal_interval[0])
normal_interval

Ширина интервала: 0.8294065944921178


(0.0, 0.8294065944921178)

Видим, что доверительный интервал стал уже. 

## Задачка 3 (бонусная)

Давайте уточним правило трёх сигм. Утвержедение: $95\%$ вероятностной массы случайной величины $X \sim N(\mu,\sigma^2)$ лежит в интервале $\mu +- c \cdot \sigma$. Чему равно значение константы $c$? 

In [10]:
# пакет для картинок
from matplotlib import pyplot as plt

# пакет для статистики
import scipy.stats as sts

# сгенерируем выборку из нормального распределения
norm_rv = sts.norm(0, 1)
sample = norm_rv.rvs(10000)

plt.hist(sample)
plt.show()

<Figure size 640x480 with 1 Axes>

In [11]:
# находим нужный нам квантиль нормального распределения
round(norm_rv.ppf((1-0.95)/2),4)

-1.96

По аналогии можно найти точное значение, на которое нужно умножать ошибку для любой константы $c$.

__ВАЖНО:__ всё, о чём написано выше, работает только тогда, когда $\bar x$ и $\hat p$ имеют нормальное распределение. Иногда это не так и нужно использовать другие распределения. Но об этом вы будете говорить подробнее на теории вероятности и математической статистике. А мы в следущей тетрадке покажем вам более универсальный метод по работе с гипотезами, работающий почти всегда. Под названием бустрап.